# 🎨 AI Meme Generator

Generate funny memes with AI! Everything you need is in this one notebook.

---

## 🚀 How to Use:

1. **Click:** Runtime → Run all
2. **Wait:** ~1 minute for setup
3. **Scroll down** to the generator
4. **Type a topic** and click "Generate Meme"
5. **Done!**

---

## 📦 Setup (Automatic)

In [ ]:
%%capture
# Install required packages
!pip install pillow requests ipywidgets

print("✅ Packages installed!")

In [ ]:
# Import libraries
import io
import os
import random
import requests
from datetime import datetime
from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as IPImage, display
import ipywidgets as widgets
from ipywidgets import interact_manual

print("✅ Libraries imported!")

## 🎨 Meme Generator Code

All the code needed to create memes:

In [ ]:
# Pre-generated captions (no API key needed!)
DEMO_CAPTIONS = {
    "working from home": [
        {"top_text": "GOING TO THE OFFICE", "bottom_text": "WORKING IN PAJAMAS"},
        {"top_text": "PROFESSIONAL ON TOP", "bottom_text": "PAJAMAS ON BOTTOM"},
    ],
    "debugging": [
        {"top_text": "FINDING THE BUG", "bottom_text": "IT WAS A MISSING SEMICOLON"},
        {"top_text": "CODE WORKS", "bottom_text": "DON'T KNOW WHY"},
    ],
    "programming": [
        {"top_text": "COPYING FROM STACKOVERFLOW", "bottom_text": "IT WORKS FIRST TRY"},
        {"top_text": "WRITING CODE", "bottom_text": "DEBUGGING CODE"},
    ],
    "monday": [
        {"top_text": "MONDAY MORNING", "bottom_text": "NEED MORE COFFEE"},
        {"top_text": "IT'S MONDAY", "bottom_text": "EVERYTHING IS FINE"},
    ],
    "online meetings": [
        {"top_text": "YOU'RE ON MUTE", "bottom_text": "STILL ON MUTE"},
        {"top_text": "CAMERA OFF", "bottom_text": "STILL IN BED"},
    ],
    "exam": [
        {"top_text": "STUDIED ALL SEMESTER", "bottom_text": "FORGOT EVERYTHING"},
        {"top_text": "EXAM IN 5 MINUTES", "bottom_text": "STARTS STUDYING NOW"},
    ],
}

# Popular meme templates
TEMPLATES = [
    {"name": "Drake Hotline Bling", "url": "https://i.imgflip.com/30b1gx.jpg"},
    {"name": "Distracted Boyfriend", "url": "https://i.imgflip.com/1ur9b0.jpg"},
    {"name": "Two Buttons", "url": "https://i.imgflip.com/1g8my4.jpg"},
    {"name": "Is This A Pigeon", "url": "https://i.imgflip.com/1o00in.jpg"},
    {"name": "Change My Mind", "url": "https://i.imgflip.com/24y43o.jpg"},
    {"name": "Buff Doge vs. Cheems", "url": "https://i.imgflip.com/43a45p.png"},
    {"name": "Running Away Balloon", "url": "https://i.imgflip.com/261o3j.jpg"},
    {"name": "UNO Draw 25 Cards", "url": "https://i.imgflip.com/3lmzyx.jpg"},
    {"name": "Expanding Brain", "url": "https://i.imgflip.com/1jwhww.jpg"},
    {"name": "Leonardo Dicaprio Cheers", "url": "https://i.imgflip.com/39t1o.jpg"},
]

def get_caption(topic):
    """Get a caption based on topic"""
    topic_lower = topic.lower()
    
    # Try exact match
    for key, captions in DEMO_CAPTIONS.items():
        if key in topic_lower:
            return random.choice(captions)
    
    # Try keywords
    if any(word in topic_lower for word in ['work', 'office', 'home', 'remote']):
        return random.choice(DEMO_CAPTIONS['working from home'])
    elif any(word in topic_lower for word in ['bug', 'debug', 'error', 'code']):
        return random.choice(DEMO_CAPTIONS['debugging'])
    elif any(word in topic_lower for word in ['meet', 'zoom', 'call']):
        return random.choice(DEMO_CAPTIONS['online meetings'])
    elif any(word in topic_lower for word in ['test', 'exam']):
        return random.choice(DEMO_CAPTIONS['exam'])
    
    # Default
    return {"top_text": topic.upper(), "bottom_text": "IT BE LIKE THAT"}

def download_image(url):
    """Download image from URL"""
    response = requests.get(url, timeout=10)
    return Image.open(io.BytesIO(response.content)).convert('RGB')

def add_text_to_image(image, top_text, bottom_text):
    """Add meme text to image"""
    draw = ImageDraw.Draw(image)
    width, height = image.size
    
    # Font size based on image height
    font_size = int(height / 10)
    
    # Try to get a good font, fallback to default
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
    except:
        font = ImageFont.load_default()
    
    # Add top text
    if top_text:
        draw_text_with_outline(draw, top_text, font, width, height * 0.05)
    
    # Add bottom text
    if bottom_text:
        bbox = draw.textbbox((0, 0), bottom_text, font=font)
        text_height = bbox[3] - bbox[1]
        draw_text_with_outline(draw, bottom_text, font, width, height * 0.95 - text_height)
    
    return image

def draw_text_with_outline(draw, text, font, width, y_pos):
    """Draw text with black outline"""
    # Calculate x position (center)
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    x = (width - text_width) / 2
    
    # Draw outline
    for adj_x in range(-2, 3):
        for adj_y in range(-2, 3):
            if adj_x != 0 or adj_y != 0:
                draw.text((x + adj_x, y_pos + adj_y), text, font=font, fill='black')
    
    # Draw main text
    draw.text((x, y_pos), text, font=font, fill='white')

def create_meme(topic):
    """Main function to create a meme"""
    if not topic.strip():
        print("⚠️ Please enter a topic!")
        return
    
    print(f"🎯 Topic: {topic}")
    print("🔍 Selecting template...")
    
    # Select random template
    template = random.choice(TEMPLATES)
    print(f"✅ Using: {template['name']}")
    
    # Get caption
    print("💬 Generating caption...")
    caption = get_caption(topic)
    print(f"   Top: {caption['top_text']}")
    print(f"   Bottom: {caption['bottom_text']}")
    
    # Download and process image
    print("🖼️ Creating meme...")
    image = download_image(template['url'])
    meme = add_text_to_image(image, caption['top_text'], caption['bottom_text'])
    
    # Save
    os.makedirs('memes', exist_ok=True)
    filename = f"memes/meme_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
    meme.save(filename, 'JPEG', quality=95)
    
    print(f"\n✅ Done!\n")
    
    # Display
    display(IPImage(filename=filename, width=500))
    
    return filename

print("✅ Meme generator ready!")

---

## 🎨 Generate Your Meme!

Enter a topic and click "Run Interact" to create a meme:

In [ ]:
interact_manual(
    create_meme,
    topic=widgets.Text(
        value='working from home',
        placeholder='Enter meme topic...',
        description='Topic:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='80%')
    )
);

---

## 💡 Try These Topics:

- working from home
- debugging code
- Monday morning
- online meetings
- exam tomorrow
- programming

Or make up your own! 🎉

---

## 📥 Download Your Memes

All memes are saved in the `memes/` folder. Download them here:

In [ ]:
# List all generated memes
import os
meme_files = [f for f in os.listdir('memes') if f.endswith('.jpg')] if os.path.exists('memes') else []

if meme_files:
    print(f"📊 You've created {len(meme_files)} memes!\n")
    print("Files:")
    for f in meme_files:
        print(f"  • {f}")
    
    # Create zip file
    import zipfile
    from datetime import datetime
    
    zip_name = f"my_memes_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip"
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for f in meme_files:
            zipf.write(os.path.join('memes', f), f)
    
    print(f"\n📦 Created: {zip_name}")
    print("\n⬇️ Download it from the Files panel (left sidebar)")
    
    # Auto-download
    from google.colab import files
    files.download(zip_name)
else:
    print("⚠️ No memes created yet! Generate some memes first.")

---

## 🎉 That's It!

You've created AI-powered memes in Google Colab!

### Features:
- ✅ No installation needed
- ✅ No API keys required
- ✅ 10 popular meme templates
- ✅ Pre-written captions
- ✅ Download your creations

### Perfect for:
- Class presentations
- Quick demos
- Having fun!

---

**Made for educational demonstrations! 🎨**